In [1]:
!git clone https://github.com/ultralytics/yolov5  # Clone YOLOv5 repository
%cd yolov5
!pip install -r requirements.txt  # Install dependencies
# !pip install --upgrade ultralytics
# !pip install --upgrade torch
# !pip install --upgrade torchvision
# !pip show yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [2]:
!pip install pyngrok

In [3]:
from pyngrok import ngrok
ngrok.set_auth_token("2oy1o89YuOJAb0YZhRxuvAMFEAz_7WeqvuofWqVBDiXWGBZUG")

In [4]:
# Install necessary packages
!pip install flask flask-ngrok ultralytics
# !pip install ultralytics==8.0.20

from flask import Flask, request, jsonify
from ultralytics import YOLO
import cv2
import numpy as np
from pyngrok import ngrok
import subprocess
import os

# Initialize Flask app
app = Flask(__name__)

# Load the YOLO model
model = YOLO("best.pt")

directory = '/content/yolov5'

directory_for_experiment_results = '/content/yolov5/runs/detect'

names = {'0': 'Sausage',
         '1': 'Egg',
         '2': 'Toast',
         '3': 'Avocado',
         '4': 'Tomato'}

def parse_results():

  files_dirs = [f for f in os.listdir(directory_for_experiment_results)]

  exp_str = sorted(files_dirs)[-1]

  full_path_results = f'{directory_for_experiment_results}/{exp_str}/labels/temp.txt'

  lines = None
  with open(full_path_results, 'r') as file:
      lines = file.readlines()

  food_count = dict()

  for line in lines:
    if(names[line.split(" ")[0]] in food_count):
      food_count[names[line.split(" ")[0]]] += 1
    else:
      food_count[names[line.split(" ")[0]]] = 1

  return food_count

@app.route("/detect", methods=["POST"])
def detect():
  if 'image' not in request.files:
    return jsonify({"error": "No image provided"}), 400

  # Save the uploaded image temporarily
  file = request.files['image']
  temp_image_path = "temp.jpg"
  with open(temp_image_path, "wb") as f:
      f.write(file.read())

  # Run detect.py with subprocess
  command = [
      "python", "detect.py", "--weights", "best.pt",
      "--source", temp_image_path, "--conf", "0.5", "--save-txt"
  ]
  process = subprocess.run(command, capture_output=True, text=True)
  if process.returncode != 0:
      return jsonify({"error": process.stderr}), 500

  # Parse results (you can adjust based on your detect.py outputs)
  return jsonify({"message": "Detection complete", "output": parse_results()})

# Start ngrok to expose the endpoint
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Run the Flask app
app.run(port=5000)

Public URL: NgrokTunnel: "https://066d-34-72-35-183.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Nov/2024 11:52:43] "POST /detect HTTP/1.1" 200 -
